Merge Sort Algorithm
===
Implementation of Merge Sort Algorithm using C for the loops


imports

In [1]:
import ctypes
import numpy as np

connect the c library. Remember using MingGW-64bits (if you are using Windows-64), and compile the library dll (os for linux) on cmd using the comand:

x86_64-w64-mingw32-gcc -shared -o clibrary.dll merge_sort.c

In [2]:
# Load the shared library into ctypes
clibrary = ctypes.CDLL('./clibrary.dll')

In [3]:
def sort(Array):
    #compute the lenght of the array to get the half of it----------------------
    lenght=len(Array)
    if lenght <= 1:
        return Array
    half=len(Array)//2

    #divide the array in two parts left and right-------------------------------
    #define the type of the data to be passed to the c function
    clibrary.left_right.argtypes = (ctypes.POINTER(ctypes.c_int), ctypes.c_int, ctypes.c_int,ctypes.POINTER(ctypes.c_int),ctypes.POINTER(ctypes.c_int)) 
    #convert the array to be passed to the c function
    arr_type = (ctypes.c_int * lenght)(*Array)
    #define two empty arrays to store the left and right part of the array
    L = (ctypes.c_int * half)()
    R = (ctypes.c_int * (lenght-half))()

    #call the c function, and automatically fill the left and right arrays because we are using pointers
    clibrary.left_right(arr_type,half,lenght,L,R)

    # convert the left and right arrays to numpy arrays
    L = np.ctypeslib.as_array(L)
    R = np.ctypeslib.as_array(R)
    
    #recursive call to the function to sort the left and right arrays
    if len(L)>1:
        L=sort(L)
    
    if len(R)>1:
        R=sort(R)
    
    #add a big number to the end of the arrays to avoid errors when merging
    L=np.append(L,100000000)
    R=np.append(R,100000000)

    #define the type of the data to be passed to the c function
    clibrary.merge_sort.argtypes = (ctypes.POINTER(ctypes.c_int),ctypes.POINTER(ctypes.c_int), ctypes.c_int, ctypes.c_int,ctypes.POINTER(ctypes.c_int))
    #empty array to store the sorted array returned by the c function
    sorted_result = (ctypes.c_int * (len(L)+len(R)-1))()
    #convert the arrays to be passed to the c function
    L = (ctypes.c_int * len(L))(*L)
    R = (ctypes.c_int * len(R))(*R)
    
    #call the c function, and automatically fill the sorted_result array because we are using pointers
    clibrary.merge_sort(R,L,half,lenght,sorted_result)

    # convert the sorted_result array to numpy array
    sorted_result = np.ctypeslib.as_array(sorted_result)
    #remove the big number added to the end of the arrays
    sorted_result=sorted_result[:-1]

    return sorted_result

Testing our implmentation

In [13]:
array_aleatorio=np.random.randint(0,100,100)
print(sort(array_aleatorio))

[ 0  1  1  1  4  5  6  8  8  8  9  9  9  9 10 11 13 13 13 16 16 18 18 24
 24 24 26 27 28 29 29 30 31 32 32 33 33 34 35 35 37 38 39 41 41 42 42 43
 44 44 45 48 48 50 50 50 52 54 55 56 57 57 58 59 61 63 63 66 66 71 72 73
 74 77 77 77 77 77 78 78 78 78 79 79 81 83 84 87 87 89 89 90 94 94 96 96
 98 98 98 99]


Comparison with numpy.sort

In [14]:
print(np.sort(array_aleatorio))

[ 0  1  1  1  4  5  6  8  8  8  9  9  9  9 10 11 13 13 13 16 16 18 18 24
 24 24 26 27 28 29 29 30 31 32 32 33 33 34 35 35 37 38 39 41 41 42 42 43
 44 44 45 48 48 50 50 50 52 54 55 56 57 57 58 59 61 63 63 66 66 71 72 73
 74 77 77 77 77 77 78 78 78 78 79 79 81 83 84 87 87 89 89 90 94 94 96 96
 98 98 98 99]
